In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class ModelTrainer:
    def __init__(self, model):
        self.model = model

    def train(self, X_train, X_test, y_train, y_test):
        self.model.fit(X_train, y_train)
        predictions = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        return accuracy

# Example usage
if __name__ == "__main__":
    # Assuming you have data X and y ready for training
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Random Forest Classifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_trainer = ModelTrainer(rf_model)
    rf_accuracy = rf_trainer.train(X_train, X_test, y_train, y_test)
    print("Random Forest Accuracy:", rf_accuracy)

    # SVM Classifier
    svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
    svm_trainer = ModelTrainer(svm_model)
    svm_accuracy = svm_trainer.train(X_train, X_test, y_train, y_test)
    print("SVM Accuracy:", svm_accuracy)
